In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import holidays
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv', index_col='row_id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv', index_col='row_id')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv', index_col='row_id')
gdp=pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
train.head()

In [ ]:
train['country'].unique()

In [ ]:
gdp=gdp.T.rename(columns={0:2015, 1:2016, 2:2017, 3:2018, 4:2019}).reset_index()

In [ ]:
gdp['country']=gdp['index'].apply(lambda x: 'Finland' if x=='GDP_Finland' else 
                   ('Norway' if x=='GDP_Norway' else 'Sweden'))

In [ ]:
gdp=gdp.iloc[1:,:]

In [ ]:
gdp.drop('index', axis=1, inplace=True)

In [ ]:
gdp=gdp.melt(id_vars=['country'], var_name='year', value_name='gdp')

In [ ]:
train['date']=pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

In [ ]:
train['year']=train['date'].dt.year
test['year']=test['date'].dt.year

In [ ]:
train=train.merge(gdp, on=['country','year'])
test=test.merge(gdp, on=['country','year'])

In [ ]:
#train['num_sold']=np.log1p(train['num_sold'])

In [ ]:
#ref: https://www.kaggle.com/shoytovma/tps-jan-2022-base-line-model
def feature_engg(df):
    #df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['week']=df['date'].dt.isocalendar().week     
    df['day']=df['date'].dt.day
    df['quarter'] = df['date'].dt.quarter
    df['dayofweek']=df['date'].dt.dayofweek
    df['is_month_start'] = df['date'].dt.is_month_start
    df['is_month_end'] = df['date'].dt.is_month_end
    df['day_of_year'] = df['date'].dt.day_of_year
    df['week_of_year']=df['date'].dt.isocalendar().week
    df['holiday']=[int(date in holidays.CountryHoliday(country, year))
                    for date, country, year
                   in zip(df['date'], df['country'], df['year'])]
    #df.drop('date', axis=1, inplace=True)
    
    return df
    
    

In [ ]:
train=feature_engg(train)

In [ ]:
test=feature_engg(test)

In [ ]:
test.drop('date', axis=1, inplace=True)

In [ ]:
#np.exp(np.log1p(train['num_sold'])).hist()

In [ ]:
print(train.shape), print(test.shape)

In [ ]:
list(train['product'].unique())

In [ ]:
from sklearn.preprocessing import FunctionTransformer


def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [ ]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [ ]:
X=train.drop(["num_sold","date"], axis=1)
y=train['num_sold']

In [ ]:
#ref:https://www.kaggle.com/yamqwe/tutorial-let-s-talk-time-series-validation?scriptVersionId=85038349
#ref: https://www.kaggle.com/jorijnsmit/found-the-holy-grail-grouptimeseriessplit
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
gss= GroupTimeSeriesSplit(n_splits=3)

In [ ]:
train['date'].dt.year.unique()

In [ ]:
#for train_idx, test_idx in gss:
    #print(train.loc[train_idx, 'date'].unique())
    #print(train.loc[test_idx, 'date'].unique())

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=3,
    
    #max_train_size=20000
    
)

In [ ]:
from sklearn.model_selection import GroupKFold

gk_fold=GroupKFold(n_splits=4).get_n_splits(groups=train.date.dt.year)

In [ ]:
#for train_idx, test_idx in gss:
    #print(train.loc[train_idx, 'date'].unique())
    #print(train.loc[test_idx, 'date'].unique())

Ref: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html#sphx-glr-auto-examples-applications-plot-cyclical-feature-engineering-py

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.experimental    import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer

categorical_columns = [
    "country",
    "store",
    "product"
    ]
categories = [
    ['Finland', 'Norway', 'Sweden'],
    ['KaggleMart', 'KaggleRama'],
    ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']
    ]

ordinal_encoder = OrdinalEncoder(categories=categories)
ohe=OneHotEncoder()
gbrt_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("categorical", ordinal_encoder, categorical_columns),
             ("month_sin", sin_transformer(12), ["month"]),
            ("month_cos", cos_transformer(12), ["month"]),
            ("day_sin", sin_transformer(7), ["day"]),
            ("day_cos", cos_transformer(7), ["day"]),
            ("week_of_yr_sin", sin_transformer(52), ["week_of_year"]),
            ("week_of_yr_cos", cos_transformer(52), ["week_of_year"]),
            
        ],
        remainder='passthrough',
    ),
    #('scaler', StandardScaler()),
    
   #HistGradientBoostingRegressor(random_state=1
   # ),
)

In [ ]:
param_grid={#"histgradientboostingregressor__learning_rate":[0.001,0.01,0.1,1],
            "histgradientboostingregressor__learning_rate":[0.1],
            #"histgradientboostingregressor__max_depth":np.arange(1,10,1),
            "histgradientboostingregressor__max_depth":[6],
            #"histgradientboostingregressor__min_samples_leaf": np.arange(5,100,10),
            "histgradientboostingregressor__min_samples_leaf": [25],
            #"histgradientboostingregressor__n_estimators": [500],
            
            
    
}

In [ ]:

custom_scorer = make_scorer(smape, greater_is_better=False)

In [ ]:
scores=[]
y_valid_probs=[]
y_preds=[]

kfold=StratifiedKFold(4, shuffle=True)
#Include sample data in Kfold it is present
for ids, (train_id, test_id) in enumerate(kfold.split(X, y, groups=train['date'].dt.year)):
    x_train=X.iloc[train_id]
    y_train=y[train_id]
    x_valid=X.iloc[test_id]
    y_valid=y[test_id]
    x_train=gbrt_pipeline.fit_transform(x_train)
    model=HistGradientBoostingRegressor(random_state=1, max_depth=6, min_samples_leaf=25, learning_rate=0.1)
    model.fit(x_train, y_train)
    y_valid_predict=model.predict(gbrt_pipeline.transform(x_valid))
    smpe=smape(y_valid, y_valid_predict)
    scores.append(smpe)
    y_valid_probs.append(y_valid_predict)
    #x_test=gbrt_pipeline.fit_transform(test)
    y_pred=model.predict(gbrt_pipeline.transform(test))
    y_preds.append(y_pred)
    

    
    

In [ ]:
print(y_preds[1])

In [ ]:
y_pred=sum(y_preds)/len(y_preds)

In [ ]:
#search=GridSearchCV(gbrt_pipeline, param_grid, n_jobs=-1, cv=ts_cv, scoring=custom_scorer)

In [ ]:
#search.fit(X, y)

In [ ]:
#search.best_params_, search.best_score_

In [ ]:
#y_pred=search.predict(X)

In [ ]:
#y_pred=y_preds.astype('int')
#y_pred=np.exp(y_pred).astype('int')

In [ ]:
#smape(y_pred=y_pred, y_true=y)

In [ ]:
#Lets evaluate our gradient boosting model 
#with the mean absolute error of the relative demand
#averaged across our 5 time-based cross-validation splits:

#def evaluate(model, X, y, cv):
#    cv_results = cross_validate(
#        model,
#        X,
#        y,
#        cv=cv,
#        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
#    )
#    mae = -cv_results["test_neg_mean_absolute_error"]
#    rmse = -cv_results["test_neg_root_mean_squared_error"]
#    print(
#        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
#        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
#    )



In [ ]:
#train.shape

In [ ]:
#y_test=search.predict(test)


In [ ]:
#y_test=np.exp(y_test).astype('int') 

In [ ]:

y_test=pd.DataFrame(y_pred).rename(columns={0:'num_sold'}).astype('int')



In [ ]:
sample_submission=sample_submission.reset_index()

In [ ]:
submission =pd.DataFrame({'row_id': sample_submission['row_id'],
                           'num_sold': y_test['num_sold']})  
submission.to_csv('submission.csv', index=False)